In [416]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [395]:
data=pd.read_csv("./data/Train.csv")

In [396]:
# covert features from string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    data[feature]=data[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])

In [397]:
'''
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]
'''

'\nfor x in range(121):\n    data["newtemp"+ str(x)] = data.temp.str[x]\n    data["newprecip"+ str(x)] = data.precip.str[x]\n    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]\n    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]\n    data["windspeed"+ str(x)] = data.wind_spd.str[x]\n    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]\n'

In [398]:
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

for col_name in features:
    data[col_name]=data[col_name].apply(remove_nan_values)

In [399]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

def aggregate_features2(x,col_name):
    #x["max_"+col_name]=x[col_name].apply(np.max)
    #x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    #x["std_"+col_name]=x[col_name].apply(np.std)
    #x["var_"+col_name]=x[col_name].apply(np.var)
    #x["median_"+col_name]=x[col_name].apply(np.median)
    #x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

for col_name in features:
    data=aggregate_features(data,col_name)

In [400]:
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
data.drop(features,1,inplace=True)

/var/folders/zb/xbgk1g2j3sj47pws2mbz0r500000gn/T/ipykernel_10414/4125182389.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop(features,1,inplace=True)


In [401]:
data.head()

,ID,location,target,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,...,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,...,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500
1,ID_train_1,D,79.131702,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,...,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333
2,ID_train_10,A,32.661304,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,...,0.342484,0.942500,2.420000,88.813333,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833
3,ID_train_100,A,53.850238,27.491667,16.941667,20.845273,3.080140,9.487259,19.541667,10.550000,...,0.336701,1.388333,2.469167,88.685000,87.965000,88.376915,0.161758,0.026166,88.400000,0.720000
4,ID_train_1000,A,177.418750,28.750000,17.525000,21.870732,3.286870,10.803515,20.950000,11.225000,...,0.181690,0.542917,1.796667,88.719167,88.268333,88.538104,0.118369,0.014011,88.552500,0.450833


In [402]:
data.groupby(['location', 'target']).count()

ID  max_temp  min_temp  mean_temp  std_temp  var_temp  \
location target                                                              
A        1.452619     1         1         1          1         1         1   
         1.959348     1         1         1          1         1         1   
         2.310889     1         1         1          1         1         1   
         2.464186     1         1         1          1         1         1   
         2.738636     1         1         1          1         1         1   
...                  ..       ...       ...        ...       ...       ...   
E        155.019535   1         1         1          1         1         1   
         163.806136   1         1         1          1         1         1   
         179.590909   1         1         1          1         1         1   
         201.738636   1         1         1          1         1         1   
         218.505385   1         1         1          1         1         1   

                     median_temp  ptp_temp  max_precip  min_precip  ...  \
location target                                                     ...   
A        1.452619              1         1           1           1  ...   
         1.959348              1         1           1           1  ...   
         2.310889              1         1           1           1  ...   
         2.464186              1         1           1           1  ...   
         2.738636              1         1           1           1  ...   
...                          ...       ...         ...         ...  ...   
E        155.019535            1         1           1           1  ...   
         163.806136            1         1           1           1  ...   
         179.590909            1         1           1           1  ...   
         201.738636            1         1           1           1  ...   
         218.505385            1         1           1           1  ...   

                     var_wind_spd  median_wind_spd  ptp_wind_spd  \
location target                                                    
A        1.452619               1                1             1   
         1.959348               1                1             1   
         2.310889               1                1             1   
         2.464186               1                1             1   
         2.738636               1                1             1   
...                           ...              ...           ...   
E        155.019535             1                1             1   
         163.806136             1                1             1   
         179.590909             1                1             1   
         201.738636             1                1             1   
         218.505385             1                1             1   

                     max_atmos_press  min_atmos_press  mean_atmos_press  \
location target                                                           
A        1.452619                  1                1                 1   
         1.959348                  1                1                 1   
         2.310889                  1                1                 1   
         2.464186                  1                1                 1   
         2.738636                  1                1                 1   
...                              ...              ...               ...   
E        155.019535                1                1                 1   
         163.806136                1                1                 1   
         179.590909                1                1                 1   
         201.738636                1                1                 1   
         218.505385                1                1                 1   

                     std_atmos_press  var_atmos_press  median_atmos_press  \
location target                                                             
A        1.4526

In [403]:
data['target'] = (data['target']>55).astype(int)

In [404]:
data.drop('ID', axis=1, inplace=True)
data = pd.get_dummies(data, columns=['location'], drop_first=True)
#data = data[data['location'] == 'D']
#data.drop('location', axis=1, inplace=True)

In [405]:
data.head()

,target,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,...,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,location_B,location_C,location_D,location_E
0,0,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,...,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500,0,1,0,0
1,1,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,...,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333,0,0,1,0
2,0,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,...,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833,0,0,0,0
3,0,27.491667,16.941667,20.845273,3.080140,9.487259,19.541667,10.550000,25.787,0.0,...,87.965000,88.376915,0.161758,0.026166,88.400000,0.720000,0,0,0,0
4,1,28.750000,17.525000,21.870732,3.286870,10.803515,20.950000,11.225000,0.136,0.0,...,88.268333,88.538104,0.118369,0.014011,88.552500,0.450833,0,0,0,0


In [406]:
X = data.drop('target', axis=1)
#X = data[['mean_wind_spd']]
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [407]:
'''
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred_linreg = linreg.predict(X_test)
mean_squared_error(y_test, y_pred_linreg, squared=False)
'''

'\nlinreg = LinearRegression()\nlinreg.fit(X_train, y_train)\ny_pred_linreg = linreg.predict(X_test)\nmean_squared_error(y_test, y_pred_linreg, squared=False)\n'

In [408]:
'''
treereg = DecisionTreeRegressor()
treereg.fit(X_train, y_train)
y_pred_treereg = treereg.predict(X_test)
mean_squared_error(y_test, y_pred_treereg, squared=False)
'''

'\ntreereg = DecisionTreeRegressor()\ntreereg.fit(X_train, y_train)\ny_pred_treereg = treereg.predict(X_test)\nmean_squared_error(y_test, y_pred_treereg, squared=False)\n'

In [409]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print(confusion_matrix(y_test, y_pred_logreg))
print(accuracy_score(y_test, y_pred_logreg))
y_pred_logreg_train = logreg.predict(X_train)
print(accuracy_score(y_train, y_pred_logreg_train))

[[1697  266]
 [ 860  285]]
0.6377091377091377
0.6489421607272142


/Users/denniseickhorn/neuefische/ML-Project-1/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [410]:
treeC = DecisionTreeClassifier()
treeC.fit(X_train, y_train)
y_pred_treeC = treeC.predict(X_test)
print(confusion_matrix(y_test, y_pred_treeC))
print(accuracy_score(y_test, y_pred_treeC))
y_pred_treeC_train = treeC.predict(X_train)
print(accuracy_score(y_train, y_pred_treeC_train))

[[1619  344]
 [ 281  864]]
0.7989060489060489
0.9917142627302711


In [411]:
sorted(np.array([X_train.columns, treeC.feature_importances_]).T, key= lambda t: t[1])

[array(['min_precip', 0.0], dtype=object),
 array(['median_precip', 0.0], dtype=object),
 array(['location_B', 0.0], dtype=object),
 array(['location_C', 0.0], dtype=object),
 array(['location_D', 0.0], dtype=object),
 array(['max_precip', 0.0001765537545146621], dtype=object),
 array(['ptp_precip', 0.0007515574138512258], dtype=object),
 array(['min_rel_humidity', 0.0027853369768912847], dtype=object),
 array(['max_wind_spd', 0.0032796970738235235], dtype=object),
 array(['min_atmos_press', 0.004656110737385863], dtype=object),
 array(['max_wind_dir', 0.004938221122406231], dtype=object),
 array(['ptp_wind_dir', 0.004988311516894584], dtype=object),
 array(['location_E', 0.007546572213842697], dtype=object),
 array(['max_temp', 0.008232551138581363], dtype=object),
 array(['ptp_rel_humidity', 0.00859666535339522], dtype=object),
 array(['max_rel_humidity', 0.009466539579513626], dtype=object),
 array(['ptp_atmos_press', 0.010681028042702831], dtype=object),
 array(['min_wind_dir', 0.0

In [417]:
param_grid = {'criterion': ['gini', 'entropy'],
                'max_depth': [None] + list(np.linspace(2,30,2).astype(int)),
                'min_samples_split': np.linspace(2,82,10).astype(int),
                'min_samples_leaf': np.linspace(2,82,10).astype(int)
}

gs = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 2, 30],
                         'min_samples_leaf': array([ 2, 10, 19, 28, 37, 46, 55, 64, 73, 82]),
                         'min_samples_split': array([ 2, 10, 19, 28, 37, 46, 55, 64, 73, 82])},
             scoring='accuracy')

In [440]:
#RanFor = gs.best_estimator_
RanFor = RandomForestClassifier(criterion = 'entropy', max_depth=30, random_state=42)
RanFor.fit(X_train, y_train)
y_pred_RanFor = RanFor.predict(X_test)
print(confusion_matrix(y_test, y_pred_RanFor))
print(accuracy_score(y_test, y_pred_RanFor))
y_pred_RanFor_train = RanFor.predict(X_train)
print(accuracy_score(y_train, y_pred_RanFor_train))

[[1709  254]
 [ 284  861]]
0.8268983268983269
0.9917142627302711
